# 查找历史金叉死叉日期

## 均线，移动平均线，连续n日的收盘平均数连线； ma5=五日均线；ma10=十日均线

## 金叉：短期均线【上穿】长期均线    买入信号

## 死叉：短期均线【下穿】长期均线    卖出信号

# 练习2

## 使用tushare包获取某股票的历史行情数据

In [82]:
import tushare as ts
import pandas as pd
df = ts.get_hist_data('601318', start='1990')
df.to_csv('中国平安601318.csv')

In [84]:
df_601318 = pd.read_csv('中国平安601318.csv', index_col='date', parse_dates=True)[['open', 'close', 'high', 'low']]
df_601318

,open,close,high,low
date,,,,
2020-04-17,71.10,72.40,73.32,70.79
2020-04-16,70.50,70.38,70.71,70.16
2020-04-15,71.07,70.85,71.38,70.65
2020-04-14,70.68,71.02,71.17,70.25
2020-04-13,70.45,70.13,70.79,70.00
...,...,...,...,...
2017-10-24,59.77,60.20,61.06,59.63
2017-10-23,59.73,59.88,60.18,59.20
2017-10-20,59.00,59.02,59.45,58.64


## 使用pandas包计算该股票历史数据的5日均线和60日均线

In [99]:
import numpy as np

df_601318['ma5'] = np.nan
df_601318['ma30'] = np.nan
df_601318 = df_601318.sort_index(ascending=True)

# # 计算五日均线
# for i in range(5, len(df_601318)):
#     df_601318['ma5'][i-1] = df_601318.iloc[i-5:i,:]['close'].mean()
#     pass

# # 计算30日均线
# for i in range(30, len(df_601318)):
#     df_601318['ma30'][i-1] = df_601318.iloc[i-30:i,:]['close'].mean()
#     pass
df_601318['ma5'] = df_601318['close'].rolling(5).mean()
df_601318['ma30'] = df_601318['close'].rolling(30).mean()
df_601318

,open,close,high,low,ma5,ma30
date,,,,,,
2017-10-18,58.06,58.98,59.03,57.87,NaN,NaN
2017-10-19,59.06,59.59,59.95,58.80,NaN,NaN
2017-10-20,59.00,59.02,59.45,58.64,NaN,NaN
2017-10-23,59.73,59.88,60.18,59.20,NaN,NaN
2017-10-24,59.77,60.20,61.06,59.63,59.534,NaN
...,...,...,...,...,...,...
2020-04-13,70.45,70.13,70.79,70.00,70.024,72.764000
2020-04-14,70.68,71.02,71.17,70.25,70.234,72.478667
2020-04-15,71.07,70.85,71.38,70.65,70.526,72.190667


## 使用matplotlib包可视化历史数据的收盘价和两条均线

In [69]:
df_601318[['close', 'ma5', 'ma30']][:100].plot()

## 分析输出的所有金叉日期和死叉日期

T   T T T （F） F F F T T T T     5日均值低于30日均值
Nan  F F F （F） T T T T F F F F     5日均值高于30日均值
  

In [118]:
# 去掉含有nan数据的行
df_601318 = df_601318.dropna()

gold_cross = list()  # 存储金叉日期
death_cross = list()  # 死叉
for i in range(1,len(df_601318)):
    # 金叉
    if df_601318['ma5'][i] >= df_601318['ma30'][i] and df_601318['ma5'][i-1] < df_601318['ma30'][i-1]:
        gold_cross.append(df_601318.index[i])
    # 死叉
    if df_601318['ma5'][i] <= df_601318['ma30'][i] and df_601318['ma5'][i-1] > df_601318['ma30'][i-1]:
        death_cross.append(df_601318.index[i])



## 如果我从2018年1月1日开始，初始资金为10万，金叉尽量买入，死叉全部卖出，则到今天为止，我的收益率如何？

In [176]:
money = 100000
hold = 0
sr1 = pd.Series(1, index=gold_cross)
sr2 = pd.Series(0, index=death_cross)
sr = sr1.append(sr2).sort_index()

for i in sr['2018':].index:
    price = df_601318.loc[i,:]['open']  # 金叉或死叉日期的开盘价格
    if 0 == sr[i]:  # 金叉买入
        buy = money//(price*100)  # 最多可以买几手
        money -= buy*100*price# 买完剩下的钱
        hold += buy*100# 持有
        pass
    else:  # 死叉卖出
        # 卖出全部
        money += hold*price
        hold = 0
        pass

# 计算最后持有股份值多少钱
price = df_601318['open'][-1]
money += hold*price
(money - 100000)/10000

0.5646000000000029